------
# Preporcess Population
##### 1. Assign Suburbs to SA2
##### 2. Find Subrub Population Proportion to SA2
##### 3. Predict Suburb Future Population and Density
-------

### 1. Assign Suburbs to SA2
1. Load datasets
2. Find out suburbs that belongs to 2 SA2
3. Assign Suburbs based on the area porportion
4. ...


load datasets


In [110]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [111]:
population = pd.read_csv("../data/raw/population.csv")
population.head()

,Unnamed: 0,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,ERP 2020,ERP 2021,ERP Change Amount,ERP Change Rate,Natureal increase,Internal migration,Overseas migration,Area km2,density 2021 persons/km2
0,0,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011001,Alfredton,15030,16263,1233,8.2,117,1143,-27,52.7,308.5
1,1,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011002,Ballarat,12097,11931,-166,-1.4,-43,-107,-16,12.4,963.8
2,2,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011003,Ballarat - North,24606,24720,114,0.5,8,123,-17,92.4,267.7
3,3,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011004,Ballarat - South,25639,25543,-96,-0.4,83,-150,-29,32.9,777.5
4,4,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,201011005,Buninyong,7721,7668,-53,-0.7,33,-78,-8,51.6,148.6


In [112]:
properties= pd.read_csv("../data/raw/properties_preporcessed_1.csv")
properties.head()

,Unnamed: 0,name,cost_text,rooms,type,cloest station,station_distance,cloest school,school_distance,CBD_distance,...,text,beds,bath,parking,Longitude,Latitude,SA2_CODE21,LOC_PID,LOC_NAME,SA2_MAIN16
0,0,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,"['3 Beds', '2 Baths', '2 Parking']",Apartment / Unit / Flat,30,1803.65,1374,667.06,1702.19,...,Per Week,3,2,2,144.971027,-37.831809,206051512.0,loc9901d119afda,Melbourne,206051132.0
1,1,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,"['1 Bed', '1 Bath', '1 Parking']",Apartment / Unit / Flat,176,371.68,2265,352.49,2267.13,...,per week,1,1,1,144.953247,-37.816228,206041505.0,loc9901d119afda,Melbourne,206041122.0
2,2,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,248.50,2189,401.73,1937.39,...,pw,1,1,0,144.958129,-37.812103,206041505.0,loc9901d119afda,Melbourne,206041122.0
3,3,3/27 Flinders Lane Melbourne VIC 3000,$420 per week,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,140,404.26,2193,586.61,790.13,...,per week,1,1,0,144.973365,-37.814732,206041503.0,loc9901d119afda,Melbourne,206041122.0
4,4,611/408 Lonsdale Street Melbourne VIC 3000,$350,"['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,525.08,2265,779.91,1657.95,...,NaN,1,1,0,144.960401,-37.812598,206041504.0,loc9901d119afda,Melbourne,206041122.0


In [113]:
# extract information needed
df1 = population[['SA2 code','ERP 2021']]

df2 = properties[['LOC_PID','SA2_CODE21']]
df2 = df2.fillna(0)
df2 = df2.drop_duplicates()
df2['SA2_CODE21']=df2['SA2_CODE21'].astype(int)

Find Overlap 


Some suburbs correspondes to multiple SA2s, find the area of each SA2 in the suburb.

In [114]:
suburb_shp = gpd.read_file("../data/raw/vic_localities/vic_localities.shp")
sa2_shp = gpd.read_file("../data/raw/SA2_shape/SA2_2021_AUST_GDA2020.shp")

In [115]:
# Suburb shape and area
suburb_shp = suburb_shp[['LOC_PID', 'LOC_NAME', 'geometry']].set_crs("EPSG:4326", allow_override=True)
suburb_shp['suburb_area'] = suburb_shp.geometry.area
suburb_area = suburb_shp.drop(columns='geometry')

# SA2 shape
sa2_shp = sa2_shp[sa2_shp['STE_NAME21'] == 'Victoria']
sa2_shp = sa2_shp[['SA2_CODE21', 'geometry']].set_crs("EPSG:4326", allow_override=True)
sa2_shp = sa2_shp.astype({'SA2_CODE21' : int})

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1472629128.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  suburb_shp['suburb_area'] = suburb_shp.geometry.area


In [116]:
# find suburbs that belongs to multiple SA2
agg_suburb_count = df2.groupby('LOC_PID', as_index=False).count()
agg_suburb_count = agg_suburb_count[agg_suburb_count['SA2_CODE21'] > 1]

In [117]:
# Find overlap area
suburb_groups = df2.groupby('LOC_PID')
suburb_sa2_overlap_areas = pd.DataFrame([], columns=["LOC_PID", "overlap_area"])
for suburb in agg_suburb_count['LOC_PID']:
  group = suburb_groups.get_group(suburb )
  sub_poly = suburb_shp[suburb_shp['LOC_PID'] == suburb ]
  sa2_poly = sa2_shp[sa2_shp['SA2_CODE21'].isin(group['SA2_CODE21'].tolist())]
  overlap = gpd.overlay(sub_poly, sa2_poly, how="intersection")
  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC

/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(group['SA2_CODE21'].iloc[i], overlap.geometry.area[i]) for i in range(0, len(overlap))]
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  suburb_sa2_overlap_areas = suburb_sa2_overlap_areas.append({"LOC_PID": suburb , "overlap_area": overlap_list}, ignore_index=True)
/var/folders/nf/d7csz1d90ngcp3fqh2v6jgn40000gn/T/ipykernel_66540/1295008611.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlap_list = [(

In [118]:
suburb_sa2_overlap_areas  # The overlap_area [(SA2_code, area),...]

,LOC_PID,overlap_area
0,loc00a9769647d7,"[(207011522, 0.0004422559047194018), (20701152..."
1,loc00d1503504f1,"[(212051321, 0.0009383326105464119), (21205132..."
2,loc00f0949ea0ad,"[(210041541, 0.0070176802621758246), (21004154..."
3,loc0475beb19ff3,"[(212031557, 0.0007066527528316065), (21203155..."
4,loc0a03ed3531fd,"[(208031188, 0.00029753589690431127), (2080111..."
...,...,...
117,locf5fef268f8cd,"[(213051363, 0.00030836751688239234), (2130515..."
118,locf72490d7eaa6,"[(213051362, 0.0010877856749048145), (21305136..."
119,locfb463ae5ac4b,"[(212011546, 0.0010768714737548117), (21202155..."
120,locfd8472c41cbe,"[(211011256, 0.0008236757283031328), (21101125..."


In [119]:
suburb_sa2_overlap_areas.to_csv("../data/raw/suburb_sa2_overlap_areas.csv")

In [123]:
import numpy as np
suburb = pd.read_csv("../data/raw/suburb_sa2_overlap_areas.csv", index_col=0)
pid_name = properties[['LOC_PID','LOC_NAME']].drop_duplicates()
pid_name.index - range(len(pid_name))
locnames = []

for i in range(len(suburb)):
    locname = pid_name.loc[pid_name['LOC_PID']==suburb['LOC_PID'][i],'LOC_NAME'].values[0]
    locnames.append(locname)
suburb['LOC_NAME'] = locnames


areas = []  
sa2s = []  
details = [] 
for i in range(len(suburb)): 
    lap = eval(suburb.loc[i,'overlap_area']) 
    sum_ = np.sum([j[1] for j in lap]) 
    most = 0 
    best_sa2 = None 
    detail= [] 
    for k in lap:
        r = k[1]/sum_  
        detail.append([k[0], k[1]/sum_]) 
        if r >most: 
            most= r 
            best_sa2 = k[0]
    areas.append([best_sa2, most])
    sa2s.append(best_sa2)
    details.append(detail)
suburb['area'] =areas
suburb['sa2'] = sa2s
suburb['detail']=details
suburb.to_csv("detail.csv", index=False)
suburb

,LOC_PID,overlap_area,LOC_NAME,area,sa2,detail
0,loc00a9769647d7,"[(207011522, 0.0004422559047194018), (20701152...",Kew,"[207011521, 0.5863603840172711]",207011521,"[[207011522, 0.4136396159827289], [207011521, ..."
1,loc00d1503504f1,"[(212051321, 0.0009383326105464119), (21205132...",Glen Waverley,"[212051321, 0.5441530612322659]",212051321,"[[212051321, 0.5441530612322659], [212051322, ..."
2,loc00f0949ea0ad,"[(210041541, 0.0070176802621758246), (21004154...",Sunbury,"[210041541, 0.5219906717257076]",210041541,"[[210041541, 0.5219906717257076], [210041540, ..."
3,loc0475beb19ff3,"[(212031557, 0.0007066527528316065), (21203155...",Clyde North,"[212031556, 0.6740090985595435]",212031556,"[[212031557, 0.23595609654612362], [212031556,..."
4,loc0a03ed3531fd,"[(208031188, 0.00029753589690431127), (2080111...",Cheltenham,"[208011171, 0.712222810475794]",208011171,"[[208031188, 0.28777718952420595], [208011171,..."
...,...,...,...,...,...,...
117,locf5fef268f8cd,"[(213051363, 0.00030836751688239234), (2130515...",Williams Landing,"[213051363, 0.7882889771631294]",213051363,"[[213051363, 0.7882889771631294], [213051587, ..."
118,locf72490d7eaa6,"[(213051362, 0.0010877856749048145), (21305136...",Hoppers Crossing,"[213051362, 0.6024209861155125]",213051362,"[[213051362, 0.6024209861155125], [213051361, ..."
119,locfb463ae5ac4b,"[(212011546, 0.0010768714737548117), (21202155...",Beaconsfield,"[212011546, 0.9469664841331236]",212011546,"[[212011546, 0.9469664841331236], [212021553, ..."
120,locfd8472c41cbe,"[(211011256, 0.0008236757283031328), (21101125...",Rowville,"[211011257, 0.44077160618182515]",211011257,"[[211011256, 0.3787493935632279], [211011258, ..."


In [124]:
suburb.loc[i,'sa2']

206051134

In [125]:
for i in range(len(suburb)):
    locname = suburb.loc[i,'LOC_PID']
    sa2= suburb.loc[i,'sa2']
    select_s = properties.loc[properties['LOC_PID']==locname,:]
    drop_index = select_s.loc[~select_s['SA2_CODE21'].isin([sa2])].index
    properties = properties.drop(list(drop_index))

In [126]:
properties.shape

(10148, 23)

In [127]:
df2 = properties[['LOC_PID','SA2_CODE21']]
df2 = df2.fillna(0)
df2['SA2_CODE21']=df2['SA2_CODE21'].astype(int)
df2.head()

,LOC_PID,SA2_CODE21
214,loc9901d119afda,206041117
261,loc9901d119afda,206041117
282,loc9901d119afda,206041117
473,loc9901d119afda,206041117
573,loc7024d58288dd,206041119


In [129]:
# What SA2 does each LOCNAME have
loc_res = {}
locality = list(set(df2['LOC_PID']))
for l in locality:
    sa2 = list(set(list(df2.loc[df2['LOC_PID']==l,:]['SA2_CODE21'])))
    if len(sa2)>1: 
        loc_res[l] =sa2 

In [130]:
loc_res

{}

In [131]:
len(loc_res) 

0

In [132]:
df1 = population[['SA2 code','ERP 2021']]
df1.head()

,SA2 code,ERP 2021
0,201011001,16263
1,201011002,11931
2,201011003,24720
3,201011004,25543
4,201011005,7668


Assign suburbs to SA2s

......

-----
### Find Suburb population proportion to SA2
1. Count the number of properties in suburbs
2. Use the properties proportion as the population proportion

In [133]:
# Find the LOC_PID for each SA2
result = {}
for sa in df1['SA2 code']:
    result[sa]={}
    a = list(set(df2.loc[df2['SA2_CODE21']==sa,:]['LOC_PID']))
    a_df = df2.loc[df2['SA2_CODE21']==sa,:]
    
    if len(a)>1:
        counts = 0
        counts_nums= []
        for locname in a:
          
            nums = a_df.loc[a_df['LOC_PID']==locname,:].shape[0]
            
            counts+=nums
            counts_nums.append(nums)
        ratio = list(map(lambda x:x/counts, counts_nums))
     
        for l in range(len(a)):
            result[sa][a[l]] = ratio[l]*population.loc[population['SA2 code']==sa,:]['ERP 2021'].values[0]
            
    
    elif len(a)==1:
        
        value = population.loc[population['SA2 code']==sa,:]['ERP 2021'].values[0]
        result[sa][a[0]]=value    

------
### Predict Suburb Future Population and Density


In [138]:
result

{201011001: {'loc8c852c59ffe5': 417.0,
  'loc98325a7e67bf': 5004.0,
  'loc82f5b45a790d': 10842.0},
 201011002: {'loc99690f6c580c': 2609.90625,
  'loc2c1864e3cbb6': 745.6875,
  'locb364a12c94c3': 1491.375,
  'loc76166a14c514': 5592.65625,
  'locc61135c99c81': 1491.375},
 201011003: {},
 201011004: {},
 201011005: {'loc24c93e0a473d': 2556.0, 'loc4b96f3425221': 5112.0},
 201011006: {'locb2505e62d563': 1592.0227272727273,
  'loc6a54ce63b777': 6140.659090909091,
  'loc853ce1db7b4e': 1364.590909090909,
  'loc4a26362045e1': 909.7272727272727},
 201011007: {},
 201011008: {'loceeb479c05f38': 1383.2727272727273,
  'locf293eac14028': 13141.09090909091,
  'loc6c88c792a41f': 691.6363636363636},
 201021009: {'loc88919eceab29': 2112.6666666666665,
  'loc9b1b54e515f7': 4225.333333333333},
 201021010: {'loc0d50c2dd088b': 1023.625, 'loc1dd5cc4604dc': 7165.375},
 201021011: {'loce8cc4236dbff': 5287.777777777778,
  'loc00e6e39d335b': 1057.5555555555554,
  'loc85e82c7c178e': 1057.5555555555554,
  'loc6215

In [139]:
pd.DataFrame(result).drop_duplicates()

,201011001,201011002,201011003,201011004,201011005,201011006,201011007,201011008,201021009,201021010,...,217031471,217031472,217031473,217031474,217031475,217031476,217041477,217041478,217041479,217041480
loc8c852c59ffe5,417.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loc98325a7e67bf,5004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loc82f5b45a790d,10842.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loc99690f6c580c,NaN,2609.90625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loc2c1864e3cbb6,NaN,745.68750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loc8d1eb0e83c7f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2271.666667,NaN,NaN,NaN
loc4ae321c5bbb4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2494.25,NaN,NaN
loce829f646f7ca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4988.50,NaN,NaN
loc1f73672977ce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22586.0,NaN


In [140]:
df = pd.DataFrame(result).drop_duplicates()
r2 = df.sum(axis=1)
r = pd.DataFrame(r2)
r.to_csv("../data/raw/population_for_suburb.csv")

In [141]:
r

,0
loc8c852c59ffe5,417.000000
loc98325a7e67bf,5004.000000
loc82f5b45a790d,10842.000000
loc99690f6c580c,2609.906250
loc2c1864e3cbb6,745.687500
...,...
loc8d1eb0e83c7f,2271.666667
loc4ae321c5bbb4,2494.250000
loce829f646f7ca,4988.500000
loc1f73672977ce,22586.000000
